In [4]:

from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [ ]:
import fastbook
import torch
from fastai.vision.all import *
from fastbook import *
import numpy as np
import numpy.random as nprnd



fastbook.setup_book()


In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])
txt = files[0].open().read(); txt[:75]

"Jack London's life was certainly colorful enough for a dozen films about di"

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))
tkn = Tokenizer(spacy)

(#702) ['Jack','London',"'s",'life','was','certainly','colorful','enough','for','a','dozen','films','about','different','aspects','of','him','.','Sad','to','say','though','that','what','his','life','was','used','for','in'...]


In [ ]:
if tkn:
    coll_repr(tkn('©   Fast.ai www.fast.ai/INDEX'), 31)
    txts = L(o.open().read() for o in files[:2000])
     
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    sub = first(sp([txt]))
    if sub:
        return ' '.join(sub[:40])

subword(1000)


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tmp/texts.out --vocab_size=1000 --model_prefix=tmp/spm --character_coverage=0.99999 --model_type=unigram --unk_id=9 --pad_id=-1 --bos_id=-1 --eos_id=-1 --minloglevel=2 --user_defined_symbols=▁xxunk,▁xxpad,▁xxbos,▁xxeos,▁xxfld,▁xxrep,▁xxwrep,▁xxup,▁xxmaj --hard_vocab_limit=false


"▁J ack ▁Lo n d on ' s ▁life ▁was ▁certain ly ▁co l or ful ▁enough ▁for ▁a ▁do z en ▁films ▁about ▁different ▁a spect s ▁of ▁him . ▁S a d ▁to ▁say ▁though ▁that ▁what ▁his"

In [ ]:
if tkn:
    toks = tkn(txt)

    toks200 = txts[:200].map(tkn)
    toks200[0]
    num = Numericalize()
    num.setup(toks200)
    print(coll_repr(num.vocab,20))
    nums = num(toks)[:20]; 
    print(nums)
    print(' '.join(num.vocab[o] for o in nums))
    
     

(#2016) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','i','it','in'...]
TensorText([   2,    8,  597,    8,  367,   24,  145,   25,  368, 1477,  203,   28,   12, 1207,  132,   58,    0,  598,   14,  109])
xxbos xxmaj jack xxmaj london 's life was certainly colorful enough for a dozen films about xxunk aspects of him


In [ ]:
from IPython.display import HTML
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1"
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

/tmp/ipykernel_34/3571472741.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])


"[xxbos, xxmaj, in, this, chapter, ,, we, will, go, back, over, the, example, of, classifying]"
"[movie, reviews, we, studied, in, chapter, 1]"
[]
[]
[]
[]


In [ ]:
nums200 = toks200.map(num)

dl = LMDataLoader(nums200)

In [ ]:

x,y = first(dl)
x.shape,y.shape

' '.join(num.vocab[o] for o in x[0][:20])

"xxbos xxmaj jack xxmaj london 's life was certainly colorful enough for a dozen films about xxunk aspects of him"

In [6]:

get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)
     

In [ ]:

dls_lm.show_batch(max_n=2)
     


,text,text_
0,xxbos xxmaj this was the third remake of xxup sleeping xxup with xxup the xxup xxunk xxmaj after xxup xxunk ) and xxup xxunk ) \n\n xxup xxunk was the only one which worked and was a better film \n\n xxup xxunk is directed by xxmaj abbas xxmaj mustan who sadly failed in their attempt here \n\n the story was good but the handling was n't that good and the heroine was shown too regressive and the climax too was,xxmaj this was the third remake of xxup sleeping xxup with xxup the xxup xxunk xxmaj after xxup xxunk ) and xxup xxunk ) \n\n xxup xxunk was the only one which worked and was a better film \n\n xxup xxunk is directed by xxmaj abbas xxmaj mustan who sadly failed in their attempt here \n\n the story was good but the handling was n't that good and the heroine was shown too regressive and the climax too was disappointing
1,"with some of the most hilarious techno - babble ever caught on film . xxmaj our hero is a tall , skinny , rat - faced redneck with a dubbed voice . xxmaj his sidekick is a xxunk testosterone filled she - beast with a dubbed voice . xxmaj throw in a fat waddling robot cop , a wisecracking robot , an incompetent "" indian "" janitor , and a gal who agrees to drive around for 14 hours to","some of the most hilarious techno - babble ever caught on film . xxmaj our hero is a tall , skinny , rat - faced redneck with a dubbed voice . xxmaj his sidekick is a xxunk testosterone filled she - beast with a dubbed voice . xxmaj throw in a fat waddling robot cop , a wisecracking robot , an incompetent "" indian "" janitor , and a gal who agrees to drive around for 14 hours to help"


In [ ]:
torch.device('cuda',0)
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)
     

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.017046,3.900476,0.300665,49.425991,37:09


In [13]:
from shutil import copy
import os

# Get the current directory
current_dir = os.getcwd()

learn.save(f'{current_dir}/1epoch')


Path('/notebooks/1epoch.pth')

In [ ]:
current_dir = os.getcwd()

#learn.unfreeze()
#learn.fit_one_cycle(10, 2e-3)
#learn.save('1epoch')
learn.load(f'{current_dir}/1epoch')
     

In [ ]:
learn.save_encoder(f'{current_dir}/finetuned')

In [ ]:
TEXT = "I liked this movie because,the actor Louis Tauru "
N_WORDS = 40
N_SENTENCES = 1
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

print("\n".join(preds))

i liked this movie because , the actor Louis Xxunk ( the Christian ) was the main character in the movie . This movie was very well done , here was the story and the story was all the same . The main plot line is a


In [7]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab), CategoryBlock),
    get_y=parent_label, get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)
dls_clas.vocab[1]

['neg', 'pos']

In [8]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos i thought that xxup rotj was clearly the best out of the three xxmaj star xxmaj wars movies . i find it surprising that xxup rotj is considered the weakest installment in the xxmaj trilogy by many who have voted . xxmaj to me it seemed like xxup rotj was the best because it had the most profound plot , the most suspense , surprises , most xxunk the ending ) and definitely the most episodic movie . i personally like the xxmaj empire xxmaj strikes xxmaj back a lot also but i think it is slightly less good than than xxup rotj since it was slower - moving , was not as episodic , and i just did not feel as much suspense or emotion as i did with the third movie . \n\n xxmaj it also seems like to me that after reading these surprising reviews that",pos
2,"xxbos xxmaj heavy - handed moralism . xxmaj writers using characters as mouthpieces to speak for themselves . xxmaj predictable , plodding plot points ( say that five times fast ) . a child 's imitation of xxmaj britney xxmaj spears . xxmaj this film has all the earmarks of a xxmaj lifetime xxmaj special reject . \n\n i honestly believe that xxmaj jesus xxmaj xxunk and xxmaj julia xxmaj xxunk set out to create a thought - provoking , emotional film on a tough subject , exploring the idea that things are not always black and white , that one who is a criminal by definition is not necessarily a bad human being , and that there can be extenuating circumstances , especially when one puts the well - being of a child first . xxmaj however , their earnestness ends up being channeled into preachy dialogue and trite",neg


In [ ]:
nums_sample = toks200[:10].map(num)

In [ ]:
nums_sample.map(len)

(#10) [804,87,130,159,111,713,279,163,453,795]

In [11]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,metrics=accuracy).to_fp16()

In [14]:
learn = learn.load_encoder(f'{current_dir}/finetuned')

In [15]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.425013,0.355299,0.847000,01:53


In [18]:
torch.device('cuda',0)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.339934,0.265779,0.891280,02:03


In [20]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.265756,0.222270,0.912320,02:47


In [21]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.228442,0.211406,0.917240,03:23
1,0.210640,0.219610,0.919040,03:23


In [29]:
# Make predictions on new text data
text = "that 01_intro.ipynb is a great movie!"
pred_class, pred_idx, outputs = learn.predict(text)

# Print the predicted class and outputs
print(f"Predicted class: {pred_class}; Outputs: {outputs}")

Predicted class: pos; Outputs: tensor([7.2811e-07, 1.0000e+00])


In [24]:
learn.save(f'{current_dir}/classifier')

Path('/notebooks/classifier.pth')